In [1]:
# 引入库包
%matplotlib inline
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import time
import warnings
warnings.filterwarnings('ignore')

# 训练集和测试集提取特征

In [2]:
#PassengerId:乘客编号
#Survived：存活情况（存活：1，死亡：0）
#Pclass：客舱等级
#Name：乘客姓名
#Sex：性别
#Age：年龄
#SibSp：同乘的兄弟姐妹/配偶数
#Parch：同乘的父母/小孩数
#Ticket：船票编号
#Fare：船票价格
#Cabin：客舱号
#Embarked：登船港口
df_train = pd.read_csv('data/train.csv',header=None)
df_train.columns = ['PassengerId','Survived','Pclass','Name','Sex','Age',"SibSp","Parch","Ticket","Fare","Cabin","Embarked"]
df_train.drop(index=0,inplace=True)
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,NaN,S
2,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
3,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,NaN,S
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
5,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,NaN,S


In [3]:
df_test = pd.read_csv('data/test.csv',header=None)
df_test.columns = ['PassengerId','Pclass','Name','Sex','Age',"SibSp","Parch","Ticket","Fare","Cabin","Embarked"]
df_test.drop(index=0,inplace=True)
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
2,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47,1,0,363272,7,NaN,S
3,894,2,"Myles, Mr. Thomas Francis",male,62,0,0,240276,9.6875,NaN,Q
4,895,3,"Wirz, Mr. Albert",male,27,0,0,315154,8.6625,NaN,S
5,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22,1,1,3101298,12.2875,NaN,S


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 12 columns):
PassengerId    891 non-null object
Survived       891 non-null object
Pclass         891 non-null object
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null object
SibSp          891 non-null object
Parch          891 non-null object
Ticket         891 non-null object
Fare           891 non-null object
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: object(12)
memory usage: 90.5+ KB


In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 1 to 418
Data columns (total 11 columns):
PassengerId    418 non-null object
Pclass         418 non-null object
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null object
SibSp          418 non-null object
Parch          418 non-null object
Ticket         418 non-null object
Fare           417 non-null object
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: object(11)
memory usage: 39.2+ KB


In [6]:
df_train.select_dtypes(include="object").describe().T.assign(
  missing_pct=df_train.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,unique,top,freq,missing_pct
PassengerId,891,891,784,1,0.000000
Survived,891,2,0,549,0.000000
Pclass,891,3,3,491,0.000000
Name,891,891,"Hegarty, Miss. Hanora ""Nora""",1,0.000000
Sex,891,2,male,577,0.000000
Age,714,88,24,30,0.198653
SibSp,891,7,0,608,0.000000
Parch,891,7,0,678,0.000000
Ticket,891,681,347082,7,0.000000
Fare,891,248,8.05,43,0.000000


训练集：
* 年龄缺失20%
* 客舱号缺失77%
* 登船港口缺失0.2%

In [7]:
df_test.select_dtypes(include="object").describe().T.assign(
  missing_pct=df_test.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,unique,top,freq,missing_pct
PassengerId,418,418,1108,1,0.000000
Pclass,418,3,3,218,0.000000
Name,418,418,"Loring, Mr. Joseph Holland",1,0.000000
Sex,418,2,male,266,0.000000
Age,332,79,21,17,0.205742
SibSp,418,7,0,283,0.000000
Parch,418,8,0,324,0.000000
Ticket,418,363,PC 17608,5,0.000000
Fare,417,169,7.75,21,0.002392
Cabin,91,76,B57 B59 B63 B66,3,0.782297


测试集：
* 年龄缺失20%
* 客舱号缺失78%
* 船票价格缺失0.2%

训练集删除:
* 乘客ID
* 名字
* 船票编号
* 客舱号（缺失值太多）

剩余特征：
* 存活情况（存活：1，死亡：0）
* 客舱等级
* 性别
* 年龄
* 同乘的兄弟姐妹/配偶数
* 同乘的父母/小孩数
* 船票价格
* 登船港口

In [8]:
#df_train.drop("PassengerId",1,inplace=True)#乘客ID
df_train.drop("Name",1,inplace=True)#名字
df_train.drop("Ticket",1,inplace=True)#船票编号
df_train.drop("Cabin",1,inplace=True)#客舱号（缺失值太多）

测试集删除:
* 乘客ID
* 名字
* 船票编号
* 客舱号（缺失值太多）

剩余特征：
* 存活情况（存活：1，死亡：0）
* 客舱等级
* 性别
* 年龄
* 同乘的兄弟姐妹/配偶数
* 同乘的父母/小孩数
* 船票价格
* 登船港口

In [9]:
#df_test.drop("PassengerId",1,inplace=True)#乘客ID
df_test.drop("Name",1,inplace=True)#名字
df_test.drop("Ticket",1,inplace=True)#船票编号
df_test.drop("Cabin",1,inplace=True)#客舱号（缺失值太多）

训练集删除：
* 登船港口（空行样本）
* 年龄（空行样本）

剩余特征：
* 存活情况（存活：1，死亡：0）
* 客舱等级
* 性别
* 年龄
* 同乘的兄弟姐妹/配偶数
* 同乘的父母/小孩数
* 船票价格
* 登船港口

In [10]:
#删除数据集中某行的特征
def drop_feature_null_rows(df,feature):
    df[feature] = df[feature].fillna("999")#登船港口
    null_rows_index = df[(df[feature] == "999")].index.tolist()
    return df.drop(null_rows_index)
df_train =drop_feature_null_rows(df_train,'Embarked')

(889, 9)

In [11]:
df_train =drop_feature_null_rows(df_train,'Age')

(712, 9)

In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 1 to 891
Data columns (total 9 columns):
PassengerId    712 non-null object
Survived       712 non-null object
Pclass         712 non-null object
Sex            712 non-null object
Age            712 non-null object
SibSp          712 non-null object
Parch          712 non-null object
Fare           712 non-null object
Embarked       712 non-null object
dtypes: object(9)
memory usage: 55.6+ KB


测试集缺失值处理：
* 按照一二三等舱各自的均价来填充

In [13]:
df_test['Fare'] = df_test['Fare'].astype("float")
df_test['Fare'] = df_test[['Fare']].fillna(df_test.groupby('Pclass').transform(np.mean))

In [14]:
df_test.select_dtypes(include="object").describe().T.assign(
  missing_pct=df_test.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,unique,top,freq,missing_pct
PassengerId,418,418,1108,1,0.000000
Pclass,418,3,3,218,0.000000
Sex,418,2,male,266,0.000000
Age,332,79,21,17,0.205742
SibSp,418,7,0,283,0.000000
Parch,418,8,0,324,0.000000
Embarked,418,3,S,270,0.000000


In [15]:
df_train['Fare'] = df_train['Fare'].astype("float")
df_train.select_dtypes(include="object").describe().T.assign(
  missing_pct=df_train.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,unique,top,freq,missing_pct
PassengerId,712,712,349,1,0.0
Survived,712,2,0,424,0.0
Pclass,712,3,3,355,0.0
Sex,712,2,male,453,0.0
Age,712,88,24,30,0.0
SibSp,712,6,0,469,0.0
Parch,712,7,0,519,0.0
Embarked,712,3,S,554,0.0


In [16]:
def fun_SibSp(x):
    x = int(x)
    if x == 8:
        return 5
    else:
        return x

df_test['SibSp']= df_test['SibSp'].apply(lambda x: fun_SibSp(x))
df_train['SibSp']= df_train['SibSp'].apply(lambda x: fun_SibSp(x))

In [17]:
def fun_Parch(x):
    x = int(x)
    if x == 9:
        return 6
    else:
        return x
df_test['Parch']= df_test['Parch'].apply(lambda x: fun_Parch(x))
df_train['Parch']= df_train['Parch'].apply(lambda x: fun_Parch(x))

In [18]:
df_test.SibSp.unique()

array([0, 1, 2, 3, 4, 5], dtype=int64)

In [19]:
df_test.Parch.unique()

array([0, 1, 3, 2, 4, 6, 5], dtype=int64)

In [20]:
df_test.select_dtypes(include="int64").describe().T.assign(
  missing_pct=df_test.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,mean,std,min,25%,50%,75%,max,missing_pct
SibSp,418.0,0.433014,0.793596,0.0,0.0,0.0,1.0,5.0,0.0
Parch,418.0,0.377990,0.870896,0.0,0.0,0.0,0.0,6.0,0.0


In [21]:
df_train_temp = df_train.copy()
df_test_temp = df_test.copy()

* 删除训练集标签

In [22]:
df_train_temp.drop("Survived",1,inplace=True)#删除训练集的标签

* 合并训练集和测试集提取特征

In [23]:
frames = [df_train_temp, df_test_temp]
df_all = pd.concat(frames)

In [24]:
df_all.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
1,1,3,male,22,1,0,7.2500,S
2,2,1,female,38,1,0,71.2833,C
3,3,3,female,26,0,0,7.9250,S
4,4,1,female,35,1,0,53.1000,S
5,5,3,male,35,0,0,8.0500,S


In [25]:
df_all.select_dtypes(include="object").describe().T.assign(
  missing_pct=df_all.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,unique,top,freq,missing_pct
PassengerId,1130,1130,903,1,0.000000
Pclass,1130,3,3,573,0.000000
Sex,1130,2,male,719,0.000000
Age,1044,98,24,47,0.076106
Embarked,1130,3,S,824,0.000000


In [26]:
df_all.select_dtypes(include="float").describe().T.assign(
  missing_pct=df_all.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,mean,std,min,25%,50%,75%,max,missing_pct
Fare,1130.0,34.938832,54.012205,0.0,8.05,14.75,31.6344,512.3292,0.0


## 提取单一特征：
* 客舱等级比例
* 性别比例
* 年龄分段
* 年龄分段比例
* 同乘的兄弟姐妹/配偶数比例
* 同乘的父母/小孩数比例
* 船票价格分段
* 船票价格分段比例
* 登船港口比例

In [27]:

def category_feature_proportion(df,feature,new_feature):
    def computer_p(x):
        return x/df.shape[0]
    f1 = df[[feature]] 
    f1[new_feature] = 1
    f1 = f1.groupby(feature).agg('sum').reset_index()
    f1[new_feature] = f1[new_feature].apply(computer_p)
    return f1


* 客舱等级比例

In [28]:
pclass_proportion_feature = category_feature_proportion(df_all,"Pclass","Pclass_proportion")
pclass_proportion_feature

,Pclass,Pclass_proportion
0,1,0.257522
1,2,0.235398
2,3,0.507080


* 性别比例

In [29]:
sex_proportion_feature = category_feature_proportion(df_all,"Sex","Sex_proportion")
sex_proportion_feature

,Sex,Sex_proportion
0,female,0.363717
1,male,0.636283


* 同乘的兄弟姐妹/配偶数比例

In [30]:
sibSp_proportion_feature = category_feature_proportion(df_all,"SibSp","SibSp_proportion")
sibSp_proportion_feature

,SibSp,SibSp_proportion
0,0,0.665487
1,1,0.259292
2,2,0.034513
3,3,0.014159
4,4,0.019469
5,5,0.007080


* 同乘的父母/小孩数比例

In [31]:
parch_proportion_feature = category_feature_proportion(df_all,"Parch","Parch_proportion")
parch_proportion_feature

,Parch,Parch_proportion
0,0,0.746018
1,1,0.143363
2,2,0.089381
3,3,0.007080
4,4,0.005310
5,5,0.005310
6,6,0.003540


* 登船港口比例

In [32]:
embarked_proportion_feature = category_feature_proportion(df_all,"Embarked","Embarked_proportion")
embarked_proportion_feature

,Embarked,Embarked_proportion
0,C,0.205310
1,Q,0.065487
2,S,0.729204


* 年龄分段

In [33]:
#年龄处理，切段
def age_process(age):
    age = int(age)
    if age<10:
        return 1
    elif age<20:
        return 2
    elif age<30:
        return 3
    elif age<40:
        return 4
    elif age<50:
        return 5
    elif age<60:
        return 6
    elif age<70:
        return 7
    else:
        return 8
    
#df_all['Age'] = df_all['Age'].astype('float')
#df_all['Age_section'] = df_all['Age'].apply(age_process)

* 年龄分段比例

In [34]:
#age_section_proportion_feature = category_feature_proportion(df_all,"Age_section","Age_section_proportion")
#age_section_proportion_feature

* 船票价格分段

In [35]:
#船票价格处理，切段
def fare_process(fare):
    fare = float(fare)
    if fare<64:
        return 1
    elif fare<128:
        return 2
    elif fare<192:
        return 3
    elif fare<256:
        return 4
    elif fare<320:
        return 5
    elif fare<448:
        return 6
    elif fare<512:
        return 7
    else:
        return 8
df_all['Fare'] = df_all['Fare'].astype('float')
df_all['Fare_section'] = df_all['Fare'].apply(fare_process)

* 船票价格分段比例

In [36]:
fare_section_proportion_feature = category_feature_proportion(df_all,"Fare_section","Fare_section_proportion")
fare_section_proportion_feature

,Fare_section,Fare_section_proportion
0,1,0.859292
1,2,0.084956
2,3,0.023894
3,4,0.016814
4,5,0.011504
5,8,0.003540


In [37]:
df_all.select_dtypes(include="object").describe().T.assign(
  missing_pct=df_all.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,unique,top,freq,missing_pct
PassengerId,1130,1130,903,1,0.000000
Pclass,1130,3,3,573,0.000000
Sex,1130,2,male,719,0.000000
Age,1044,98,24,47,0.076106
Embarked,1130,3,S,824,0.000000


In [38]:
df_all.select_dtypes(include="float").describe().T.assign(
  missing_pct=df_all.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,mean,std,min,25%,50%,75%,max,missing_pct
Fare,1130.0,34.938832,54.012205,0.0,8.05,14.75,31.6344,512.3292,0.0


In [39]:
df_all.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Fare_section
1,1,3,male,22,1,0,7.2500,S,1
2,2,1,female,38,1,0,71.2833,C,2
3,3,3,female,26,0,0,7.9250,S,1
4,4,1,female,35,1,0,53.1000,S,1
5,5,3,male,35,0,0,8.0500,S,1


In [40]:
df_all = pd.merge(df_all, pclass_proportion_feature, on=["Pclass"], how='left')
df_all = pd.merge(df_all, sex_proportion_feature, on=["Sex"], how='left')
df_all = pd.merge(df_all, sibSp_proportion_feature, on=["SibSp"], how='left')
df_all = pd.merge(df_all, parch_proportion_feature, on=["Parch"], how='left')
df_all = pd.merge(df_all, embarked_proportion_feature, on=["Embarked"], how='left')
#df_all = pd.merge(df_all, age_section_proportion_feature, on=["Age_section"], how='left')
df_all = pd.merge(df_all, fare_section_proportion_feature, on=["Fare_section"], how='left')

In [41]:
df_all.select_dtypes(include="float").describe().T.assign(
  missing_pct=df_all.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,mean,std,min,25%,50%,75%,max,missing_pct
Fare,1130.0,34.938832,54.012205,0.000000,8.050000,14.750000,31.634400,512.329200,0.0
Pclass_proportion,1130.0,0.378860,0.130337,0.235398,0.257522,0.507080,0.507080,0.507080,0.0
Sex_proportion,1130.0,0.537146,0.131181,0.363717,0.363717,0.636283,0.636283,0.636283,0.0
SibSp_proportion,1130.0,0.511926,0.224024,0.007080,0.259292,0.665487,0.665487,0.665487,0.0
Parch_proportion,1130.0,0.585203,0.276508,0.003540,0.143363,0.746018,0.746018,0.746018,0.0
Embarked_proportion,1130.0,0.578178,0.249890,0.065487,0.205310,0.729204,0.729204,0.729204,0.0
Fare_section_proportion,1130.0,0.746599,0.278888,0.003540,0.859292,0.859292,0.859292,0.859292,0.0


In [42]:
df_all.select_dtypes(include="object").describe().T.assign(
  missing_pct=df_all.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,unique,top,freq,missing_pct
PassengerId,1130,1130,903,1,0.000000
Pclass,1130,3,3,573,0.000000
Sex,1130,2,male,719,0.000000
Age,1044,98,24,47,0.076106
Embarked,1130,3,S,824,0.000000


In [43]:
df_all.columns

Index(['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'Fare_section', 'Pclass_proportion', 'Sex_proportion',
       'SibSp_proportion', 'Parch_proportion', 'Embarked_proportion',
       'Fare_section_proportion'],
      dtype='object')

* 归一化+标准化

In [44]:
from sklearn import preprocessing

#归一化处理
min_max_scaler = preprocessing.MinMaxScaler()
X_temp = min_max_scaler.fit_transform(df_all[['SibSp', 'Parch', 'Fare',
        'Pclass_proportion', 'Sex_proportion',
        'SibSp_proportion', 'Parch_proportion', 'Embarked_proportion',
        'Fare_section_proportion']])
#标准化处理
df_all[['SibSp', 'Parch', 'Fare',
        'Pclass_proportion', 'Sex_proportion',
        'SibSp_proportion', 'Parch_proportion', 'Embarked_proportion',
        'Fare_section_proportion']] = preprocessing.scale(X_temp)

* One-Hot

In [45]:
#One-Hot Encoding
df_all = pd.get_dummies(df_all,columns = ['Pclass', 'Sex', 'Fare_section', 'Embarked'])

In [46]:
df_all.head()

,PassengerId,Age,SibSp,Parch,Fare,Pclass_proportion,Sex_proportion,SibSp_proportion,Parch_proportion,Embarked_proportion,...,Sex_male,Fare_section_1,Fare_section_2,Fare_section_3,Fare_section_4,Fare_section_5,Fare_section_8,Embarked_C,Embarked_Q,Embarked_S
0,1,22,0.584568,-0.4795,-0.512867,0.984190,0.756060,-1.128208,0.581848,0.604633,...,1,1,0,0,0,0,0,0,0,1
1,2,38,0.584568,-0.4795,0.673192,-0.931363,-1.322646,-1.128208,0.581848,-1.492790,...,0,0,1,0,0,0,0,1,0,0
2,3,26,-0.548472,-0.4795,-0.500365,0.984190,-1.322646,0.685770,0.581848,0.604633,...,0,1,0,0,0,0,0,0,0,1
3,4,35,0.584568,-0.4795,0.336391,-0.931363,-1.322646,-1.128208,0.581848,0.604633,...,0,1,0,0,0,0,0,0,0,1
4,5,35,-0.548472,-0.4795,-0.498049,0.984190,0.756060,0.685770,0.581848,0.604633,...,1,1,0,0,0,0,0,0,0,1


* 划分训练集和测试集

In [47]:
age_not_null = df_all[df_all['Age'].notnull()]
age_is_null = df_all[df_all['Age'].isnull()]

In [48]:
Y = age_not_null.Age
X = age_not_null.drop('Age',1,inplace=False)
X_test = age_is_null.drop('Age',1,inplace=False)

In [49]:
X.columns

Index(['PassengerId', 'SibSp', 'Parch', 'Fare', 'Pclass_proportion',
       'Sex_proportion', 'SibSp_proportion', 'Parch_proportion',
       'Embarked_proportion', 'Fare_section_proportion', 'Pclass_1',
       'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Fare_section_1',
       'Fare_section_2', 'Fare_section_3', 'Fare_section_4', 'Fare_section_5',
       'Fare_section_8', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [50]:
#训练集去除“PassengerId”
X_train = X[['SibSp', 'Parch', 'Fare', 'Pclass_proportion',
       'Sex_proportion', 'SibSp_proportion', 'Parch_proportion',
       'Embarked_proportion', 'Fare_section_proportion', 'Pclass_1',
       'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Fare_section_1',
       'Fare_section_2', 'Fare_section_3', 'Fare_section_4', 'Fare_section_5',
       'Fare_section_8', 'Embarked_C', 'Embarked_Q', 'Embarked_S']]
X_pred = X_test[['SibSp', 'Parch', 'Fare', 'Pclass_proportion',
       'Sex_proportion', 'SibSp_proportion', 'Parch_proportion',
       'Embarked_proportion', 'Fare_section_proportion', 'Pclass_1',
       'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Fare_section_1',
       'Fare_section_2', 'Fare_section_3', 'Fare_section_4', 'Fare_section_5',
       'Fare_section_8', 'Embarked_C', 'Embarked_Q', 'Embarked_S']]

In [51]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()#多元线性回归模型
linreg.fit(X_train, Y)#数据拟合
y_pred = linreg.predict(X_pred)

In [52]:
X_test['Age'] = y_pred

In [53]:
X_test.head()

,PassengerId,SibSp,Parch,Fare,Pclass_proportion,Sex_proportion,SibSp_proportion,Parch_proportion,Embarked_proportion,Fare_section_proportion,...,Fare_section_1,Fare_section_2,Fare_section_3,Fare_section_4,Fare_section_5,Fare_section_8,Embarked_C,Embarked_Q,Embarked_S,Age
722,902,-0.548472,-0.479500,-0.500905,0.984190,0.756060,0.685770,0.581848,0.604633,0.404259,...,1,0,0,0,0,0,0,0,1,27.846132
734,914,-0.548472,-0.479500,-0.060301,-0.931363,-1.322646,0.685770,0.581848,0.604633,0.404259,...,1,0,0,0,0,0,0,0,1,41.730688
741,921,1.717609,-0.479500,-0.245602,0.984190,0.756060,-2.132021,0.581848,-1.492790,0.404259,...,1,0,0,0,0,0,1,0,0,23.069216
745,925,0.584568,1.845972,-0.212802,0.984190,-1.322646,-1.128208,-1.793953,0.604633,0.404259,...,1,0,0,0,0,0,0,0,1,21.220567
748,928,-0.548472,-0.479500,-0.498049,0.984190,-1.322646,0.685770,0.581848,0.604633,0.404259,...,1,0,0,0,0,0,0,0,1,25.401400


In [54]:
pred_age_feature = X_test[["PassengerId","Age"]]
pred_age_feature.shape

(86, 2)

In [55]:
pred_age_feature.describe().T.assign(
  missing_pct=pred_age_feature.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,mean,std,min,25%,50%,75%,max,missing_pct
Age,86.0,29.12004,6.530425,3.527584,26.177942,27.879762,30.65337,47.356674,0.0


In [56]:
age_is_null = df_test_temp[df_test_temp['Age'].isnull()]
age_not_null = df_test_temp[df_test_temp['Age'].notnull()]

print(age_not_null.shape)

age_is_null.drop('Age',1,inplace=True)
print(age_is_null.shape)

age_is_null = pd.merge(age_is_null, pred_age_feature, on=["PassengerId"], how='left')

frames = [age_is_null, age_not_null]
df_test = pd.concat(frames)


(332, 8)
(86, 7)


In [57]:
df_test['Age'] = df_test['Age'].astype('float')
df_test.select_dtypes(include="object").describe().T.assign(
  missing_pct=df_test.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,unique,top,freq,missing_pct
Embarked,418,3,S,270,0.0
PassengerId,418,418,1108,1,0.0
Pclass,418,3,3,218,0.0
Sex,418,2,male,266,0.0


In [58]:
df_test.head()

,Age,Embarked,Fare,Parch,PassengerId,Pclass,Sex,SibSp
0,27.846132,S,7.8958,0,902,3,male,0
1,41.730688,S,31.6833,0,914,1,female,0
2,23.069216,C,21.6792,0,921,3,male,2
3,21.220567,S,23.4500,2,925,3,female,1
4,25.401400,S,8.0500,0,928,3,female,0


In [59]:
df_train['Age'] = df_train['Age'].astype('float')
df_train.select_dtypes(include="object").describe().T.assign(
  missing_pct=df_train.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,unique,top,freq,missing_pct
PassengerId,712,712,349,1,0.0
Survived,712,2,0,424,0.0
Pclass,712,3,3,355,0.0
Sex,712,2,male,453,0.0
Embarked,712,3,S,554,0.0


In [60]:
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
1,1,0,3,male,22.0,1,0,7.2500,S
2,2,1,1,female,38.0,1,0,71.2833,C
3,3,1,3,female,26.0,0,0,7.9250,S
4,4,1,1,female,35.0,1,0,53.1000,S
5,5,0,3,male,35.0,0,0,8.0500,S


In [61]:
df_train.select_dtypes(include="float").describe().T.assign(
  missing_pct=df_train.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,mean,std,min,25%,50%,75%,max,missing_pct
Age,712.0,29.642093,14.492933,0.42,20.00,28.00000,38.0,80.0000,0.0
Fare,712.0,34.567251,52.938648,0.00,8.05,15.64585,33.0,512.3292,0.0


In [62]:
df_train.shape

(712, 9)

In [63]:
df_test.head()

,Age,Embarked,Fare,Parch,PassengerId,Pclass,Sex,SibSp
0,27.846132,S,7.8958,0,902,3,male,0
1,41.730688,S,31.6833,0,914,1,female,0
2,23.069216,C,21.6792,0,921,3,male,2
3,21.220567,S,23.4500,2,925,3,female,1
4,25.401400,S,8.0500,0,928,3,female,0


In [64]:
df_test.select_dtypes(include="float").describe().T.assign(
  missing_pct=df_test.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,mean,std,min,25%,50%,75%,max,missing_pct
Age,418.0,30.035463,12.982371,0.17,22.0000,27.868411,37.000000,76.0000,0.0
Fare,418.0,35.571764,55.851997,0.00,7.8958,14.454200,31.471875,512.3292,0.0


In [65]:
df_test.shape

(418, 8)

## 提取组合特征-幸存率：
* 客舱等级&幸存比例
* 性别&幸存比例
* 分段年龄&幸存比例
* 同乘的兄弟姐妹/配偶数&幸存率比例
* 同乘的父母/小孩数&幸存率比例
* 船票价格分段&幸存率比例
* 登船港口&幸存率比例

In [66]:
def extrac_group_feature(df,feature1,feature2,new_feature):
    def computer_p(x):
        return x/df.shape[0]
    t1 = df[[feature1,feature2]]
    t1[new_feature] = 1
    t1 = t1.groupby([feature1,feature2]).agg('sum').reset_index()
    t1[new_feature] = t1[new_feature].apply(computer_p)
    return t1

* 客舱等级&幸存比例

In [67]:
pclass_survived_proportion_feature= extrac_group_feature(df_train,"Pclass","Survived","pclass_survived_proportion")
pclass_survived_proportion_feature.drop("Survived",1,inplace=True)
pclass_survived_proportion_feature = pclass_survived_proportion_feature.groupby(["Pclass"]).agg('sum').reset_index()
pclass_survived_proportion_feature

,Pclass,pclass_survived_proportion
0,1,0.258427
1,2,0.242978
2,3,0.498596


* 性别&幸存比例

In [68]:
sex_survived_proportion_feature= extrac_group_feature(df_train,"Sex","Survived","sex_survived_proportion")
sex_survived_proportion_feature.drop("Survived",1,inplace=True)
sex_survived_proportion_feature = sex_survived_proportion_feature.groupby(["Sex"]).agg('sum').reset_index()
sex_survived_proportion_feature

,Sex,sex_survived_proportion
0,female,0.363764
1,male,0.636236


* 分段年龄&幸存比例

In [69]:
df_train['Age'] = df_train['Age'].astype('float')
df_train['Age_section'] = df_train['Age'].apply(age_process)

In [70]:
age_section_survived_proportion_feature= extrac_group_feature(df_train,"Age_section","Survived","age_section_survived_proportion")
age_section_survived_proportion_feature.drop("Survived",1,inplace=True)
age_section_survived_proportion_feature = age_section_survived_proportion_feature.groupby(["Age_section"]).agg('sum').reset_index()
age_section_survived_proportion_feature

,Age_section,age_section_survived_proportion
0,1,0.087079
1,2,0.143258
2,3,0.308989
3,4,0.233146
4,5,0.125000
5,6,0.067416
6,7,0.025281
7,8,0.009831


* 同乘的兄弟姐妹/配偶数&幸存率比例

In [71]:
sibSp_survived_proportion_feature = extrac_group_feature(df_train,"SibSp","Survived","sibSp_survived_proportion")
sibSp_survived_proportion_feature.drop("Survived",1,inplace=True)
sibSp_survived_proportion_feature = sibSp_survived_proportion_feature.groupby(["SibSp"]).agg('sum').reset_index()
sibSp_survived_proportion_feature

,SibSp,sibSp_survived_proportion
0,0,0.658708
1,1,0.257022
2,2,0.035112
3,3,0.016854
4,4,0.025281
5,5,0.007022


* 同乘的父母/小孩数&幸存率比例

In [72]:
parch_survived_proportion_feature = extrac_group_feature(df_train,"Parch","Survived","parch_survived_proportion")
parch_survived_proportion_feature.drop("Survived",1,inplace=True)
parch_survived_proportion_feature = parch_survived_proportion_feature.groupby(["Parch"]).agg('sum').reset_index()
parch_survived_proportion_feature

,Parch,parch_survived_proportion
0,0,0.728933
1,1,0.154494
2,2,0.095506
3,3,0.007022
4,4,0.005618
5,5,0.007022
6,6,0.001404


* 船票价格分段&幸存率比例

In [73]:
df_train['Fare'] = df_train['Fare'].astype('float')
df_train['Fare_section'] = df_train['Fare'].apply(fare_process)

In [74]:
fare_section_survived_proportion_feature = extrac_group_feature(df_train,"Fare_section","Survived","fare_section_survived_proportion")
fare_section_survived_proportion_feature.drop("Survived",1,inplace=True)
fare_section_survived_proportion_feature = fare_section_survived_proportion_feature.groupby(["Fare_section"]).agg('sum').reset_index()
fare_section_survived_proportion_feature

,Fare_section,fare_section_survived_proportion
0,1,0.858146
1,2,0.094101
2,3,0.022472
3,4,0.012640
4,5,0.008427
5,8,0.004213


* 登船港口&幸存率比例

In [75]:
embarked_survived_proportion_feature = extrac_group_feature(df_train,"Embarked","Survived","embarked_survived_proportion")
embarked_survived_proportion_feature.drop("Survived",1,inplace=True)
embarked_survived_proportion_feature = embarked_survived_proportion_feature.groupby(["Embarked"]).agg('sum').reset_index()
embarked_survived_proportion_feature

,Embarked,embarked_survived_proportion
0,C,0.182584
1,Q,0.039326
2,S,0.778090


* 客舱等级&幸存比例
* 性别&幸存比例
* 分段年龄&幸存比例
* 同乘的兄弟姐妹/配偶数&幸存率比例
* 同乘的父母/小孩数&幸存率比例
* 船票价格分段&幸存率比例
* 登船港口&幸存率比例

In [76]:
df_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare', 'Embarked', 'Age_section', 'Fare_section'],
      dtype='object')

In [77]:
df_train.shape

(712, 11)

In [78]:
df_test['Age'] = df_test['Age'].astype('float')
df_test['Age_section'] = df_test['Age'].apply(age_process)
df_test['Fare'] = df_test['Fare'].astype('float')
df_test['Fare_section'] = df_test['Fare'].apply(fare_process)

In [79]:
df_test.columns

Index(['Age', 'Embarked', 'Fare', 'Parch', 'PassengerId', 'Pclass', 'Sex',
       'SibSp', 'Age_section', 'Fare_section'],
      dtype='object')

In [80]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 416
Data columns (total 10 columns):
Age             418 non-null float64
Embarked        418 non-null object
Fare            418 non-null float64
Parch           418 non-null int64
PassengerId     418 non-null object
Pclass          418 non-null object
Sex             418 non-null object
SibSp           418 non-null int64
Age_section     418 non-null int64
Fare_section    418 non-null int64
dtypes: float64(2), int64(4), object(4)
memory usage: 35.9+ KB


In [81]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 1 to 891
Data columns (total 11 columns):
PassengerId     712 non-null object
Survived        712 non-null object
Pclass          712 non-null object
Sex             712 non-null object
Age             712 non-null float64
SibSp           712 non-null int64
Parch           712 non-null int64
Fare            712 non-null float64
Embarked        712 non-null object
Age_section     712 non-null int64
Fare_section    712 non-null int64
dtypes: float64(2), int64(4), object(5)
memory usage: 66.8+ KB


In [82]:
#提取单一特征
df_train = pd.merge(df_train, pclass_proportion_feature, on=["Pclass"], how='left')
df_train = pd.merge(df_train, sex_proportion_feature, on=["Sex"], how='left')
df_train = pd.merge(df_train, sibSp_proportion_feature, on=["SibSp"], how='left')
df_train = pd.merge(df_train, parch_proportion_feature, on=["Parch"], how='left')
df_train = pd.merge(df_train, embarked_proportion_feature, on=["Embarked"], how='left')
df_train = pd.merge(df_train, fare_section_proportion_feature, on=["Fare_section"], how='left')

#提取组合特征
df_train = pd.merge(df_train, pclass_survived_proportion_feature, on=["Pclass"], how='left')
df_train = pd.merge(df_train, sex_survived_proportion_feature, on=["Sex"], how='left')
df_train = pd.merge(df_train, age_section_survived_proportion_feature, on=["Age_section"], how='left')
df_train = pd.merge(df_train, sibSp_survived_proportion_feature, on=["SibSp"], how='left')
df_train = pd.merge(df_train, parch_survived_proportion_feature, on=["Parch"], how='left')
df_train = pd.merge(df_train, fare_section_survived_proportion_feature, on=["Fare_section"], how='left')
df_train = pd.merge(df_train, embarked_survived_proportion_feature, on=["Embarked"], how='left')

In [83]:
df_train.select_dtypes(include="float").describe().T.assign(
  missing_pct=df_train.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,mean,std,min,25%,50%,75%,max,missing_pct
Age,712.0,29.642093,14.492933,0.420000,20.000000,28.000000,38.000000,80.000000,0.0
Fare,712.0,34.567251,52.938648,0.000000,8.050000,15.645850,33.000000,512.329200,0.0
Pclass_proportion,712.0,0.376575,0.130466,0.235398,0.257522,0.257522,0.507080,0.507080,0.0
Sex_proportion,712.0,0.537133,0.131219,0.363717,0.363717,0.636283,0.636283,0.636283,0.0
SibSp_proportion,712.0,0.506998,0.228223,0.007080,0.259292,0.665487,0.665487,0.665487,0.0
Parch_proportion,712.0,0.574604,0.282070,0.003540,0.143363,0.746018,0.746018,0.746018,0.0
Embarked_proportion,712.0,0.607448,0.229535,0.065487,0.729204,0.729204,0.729204,0.729204,0.0
Fare_section_proportion,712.0,0.746254,0.278480,0.003540,0.859292,0.859292,0.859292,0.859292,0.0
pclass_survived_proportion,712.0,0.374420,0.124035,0.242978,0.258427,0.258427,0.498596,0.498596,0.0
sex_survived_proportion,712.0,0.537120,0.131173,0.363764,0.363764,0.636236,0.636236,0.636236,0.0


In [84]:
#提取单一特征
df_test = pd.merge(df_test, pclass_proportion_feature, on=["Pclass"], how='left')
df_test = pd.merge(df_test, sex_proportion_feature, on=["Sex"], how='left')
df_test = pd.merge(df_test, sibSp_proportion_feature, on=["SibSp"], how='left')
df_test = pd.merge(df_test, parch_proportion_feature, on=["Parch"], how='left')
df_test = pd.merge(df_test, embarked_proportion_feature, on=["Embarked"], how='left')
df_test = pd.merge(df_test, fare_section_proportion_feature, on=["Fare_section"], how='left')

#提取组合特征
df_test = pd.merge(df_test, pclass_survived_proportion_feature, on=["Pclass"], how='left')
df_test = pd.merge(df_test, sex_survived_proportion_feature, on=["Sex"], how='left')
df_test = pd.merge(df_test, age_section_survived_proportion_feature, on=["Age_section"], how='left')
df_test = pd.merge(df_test, sibSp_survived_proportion_feature, on=["SibSp"], how='left')
df_test = pd.merge(df_test, parch_survived_proportion_feature, on=["Parch"], how='left')
df_test = pd.merge(df_test, fare_section_survived_proportion_feature, on=["Fare_section"], how='left')
df_test = pd.merge(df_test, embarked_survived_proportion_feature, on=["Embarked"], how='left')


In [85]:
df_test.select_dtypes(include="float").describe().T.assign(
  missing_pct=df_test.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,mean,std,min,25%,50%,75%,max,missing_pct
Age,418.0,30.035463,12.982371,0.170000,22.000000,27.868411,37.000000,76.000000,0.0
Fare,418.0,35.571764,55.851997,0.000000,7.895800,14.454200,31.471875,512.329200,0.0
Pclass_proportion,418.0,0.382752,0.130182,0.235398,0.257522,0.507080,0.507080,0.507080,0.0
Sex_proportion,418.0,0.537168,0.131274,0.363717,0.363717,0.636283,0.636283,0.636283,0.0
SibSp_proportion,418.0,0.520320,0.216697,0.007080,0.259292,0.665487,0.665487,0.665487,0.0
Parch_proportion,418.0,0.603258,0.266128,0.003540,0.746018,0.746018,0.746018,0.746018,0.0
Embarked_proportion,418.0,0.528323,0.274372,0.065487,0.205310,0.729204,0.729204,0.729204,0.0
Fare_section_proportion,418.0,0.747186,0.279915,0.003540,0.859292,0.859292,0.859292,0.859292,0.0
pclass_survived_proportion,418.0,0.380245,0.123825,0.242978,0.258427,0.498596,0.498596,0.498596,0.0
sex_survived_proportion,418.0,0.537155,0.131229,0.363764,0.363764,0.636236,0.636236,0.636236,0.0


One-Hot Encoded：
* 客舱等级
* 性别
* 登船港口
* 分段年龄
* 船票价格年龄

In [86]:
df_train = pd.get_dummies(df_train,columns = ['Pclass', 'Sex',
       'Embarked', 'Age_section', 'Fare_section'])
df_train.head()

,PassengerId,Survived,Age,SibSp,Parch,Fare,Pclass_proportion,Sex_proportion,SibSp_proportion,Parch_proportion,...,Age_section_5,Age_section_6,Age_section_7,Age_section_8,Fare_section_1,Fare_section_2,Fare_section_3,Fare_section_4,Fare_section_5,Fare_section_8
0,1,0,22.0,1,0,7.2500,0.507080,0.636283,0.259292,0.746018,...,0,0,0,0,1,0,0,0,0,0
1,2,1,38.0,1,0,71.2833,0.257522,0.363717,0.259292,0.746018,...,0,0,0,0,0,1,0,0,0,0
2,3,1,26.0,0,0,7.9250,0.507080,0.363717,0.665487,0.746018,...,0,0,0,0,1,0,0,0,0,0
3,4,1,35.0,1,0,53.1000,0.257522,0.363717,0.259292,0.746018,...,0,0,0,0,1,0,0,0,0,0
4,5,0,35.0,0,0,8.0500,0.507080,0.636283,0.665487,0.746018,...,0,0,0,0,1,0,0,0,0,0


In [87]:
df_test = pd.get_dummies(df_test,columns = ['Pclass', 'Sex',
       'Embarked', 'Age_section', 'Fare_section'])
df_test.head()

,Age,Fare,Parch,PassengerId,SibSp,Pclass_proportion,Sex_proportion,SibSp_proportion,Parch_proportion,Embarked_proportion,...,Age_section_5,Age_section_6,Age_section_7,Age_section_8,Fare_section_1,Fare_section_2,Fare_section_3,Fare_section_4,Fare_section_5,Fare_section_8
0,27.846132,7.8958,0,902,0,0.507080,0.636283,0.665487,0.746018,0.729204,...,0,0,0,0,1,0,0,0,0,0
1,41.730688,31.6833,0,914,0,0.257522,0.363717,0.665487,0.746018,0.729204,...,1,0,0,0,1,0,0,0,0,0
2,23.069216,21.6792,0,921,2,0.507080,0.636283,0.034513,0.746018,0.205310,...,0,0,0,0,1,0,0,0,0,0
3,21.220567,23.4500,2,925,1,0.507080,0.363717,0.259292,0.089381,0.729204,...,0,0,0,0,1,0,0,0,0,0
4,25.401400,8.0500,0,928,0,0.507080,0.363717,0.665487,0.746018,0.729204,...,0,0,0,0,1,0,0,0,0,0


In [88]:
from sklearn import preprocessing

#归一化处理
min_max_scaler = preprocessing.MinMaxScaler()
X_temp = min_max_scaler.fit_transform(df_train[['Age', 'SibSp', 'Parch', 'Fare',
       'Pclass_proportion', 'Sex_proportion', 'SibSp_proportion',
       'Parch_proportion', 'Embarked_proportion', 'Fare_section_proportion',
       'pclass_survived_proportion', 'sex_survived_proportion',
       'age_section_survived_proportion', 'sibSp_survived_proportion',
       'parch_survived_proportion', 'fare_section_survived_proportion',
       'embarked_survived_proportion']])
#标准化处理
df_train[['Age', 'SibSp', 'Parch', 'Fare',
       'Pclass_proportion', 'Sex_proportion', 'SibSp_proportion',
       'Parch_proportion', 'Embarked_proportion', 'Fare_section_proportion',
       'pclass_survived_proportion', 'sex_survived_proportion',
       'age_section_survived_proportion', 'sibSp_survived_proportion',
       'parch_survived_proportion', 'fare_section_survived_proportion',
       'embarked_survived_proportion']] = preprocessing.scale(X_temp)


#归一化处理
min_max_scaler = preprocessing.MinMaxScaler()
X_temp = min_max_scaler.fit_transform(df_test[['Age', 'SibSp', 'Parch', 'Fare',
       'Pclass_proportion', 'Sex_proportion', 'SibSp_proportion',
       'Parch_proportion', 'Embarked_proportion', 'Fare_section_proportion',
       'pclass_survived_proportion', 'sex_survived_proportion',
       'age_section_survived_proportion', 'sibSp_survived_proportion',
       'parch_survived_proportion', 'fare_section_survived_proportion',
       'embarked_survived_proportion']])
#标准化处理
df_test[['Age', 'SibSp', 'Parch', 'Fare',
       'Pclass_proportion', 'Sex_proportion', 'SibSp_proportion',
       'Parch_proportion', 'Embarked_proportion', 'Fare_section_proportion',
       'pclass_survived_proportion', 'sex_survived_proportion',
       'age_section_survived_proportion', 'sibSp_survived_proportion',
       'parch_survived_proportion', 'fare_section_survived_proportion',
       'embarked_survived_proportion']] = preprocessing.scale(X_temp)

In [89]:
df_train.drop('PassengerId',1,inplace=True)
Y = df_train.Survived
X = df_train.drop('Survived',1,inplace=False)

In [90]:
X.shape

(712, 39)

# 最终LR 模型训练

In [91]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X, Y )

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

# 测试集预测

In [92]:
df_test_feature = df_test.copy()

In [93]:
df_test.drop("PassengerId",1,inplace=True)

In [94]:
Predict = lr.predict(df_test)

In [95]:
# submission
df = pd.DataFrame({"PassengerId": df_test_feature["PassengerId"].values, "Survived": Predict})
df.to_csv("gender_submission.csv", index=False)